In [1]:
import networkx as nx
import itertools as it
from pyvis.network import Network

We'll make some De Bruijn graphs and look at some observations.

In [2]:
n = 3
k = 2

In [3]:
alphabet = [str(_) for _ in range(k)]

In [4]:
G = nx.DiGraph()

In [5]:
nodes = [''.join(tup) for tup in list(it.product(alphabet, repeat=n))]
print(nodes)

['000', '001', '010', '011', '100', '101', '110', '111']


In [6]:
edges = []
for node in nodes:
    for letter in alphabet:
        edges.append((node, node[1:]+letter))
print(edges)

[('000', '000'), ('000', '001'), ('001', '010'), ('001', '011'), ('010', '100'), ('010', '101'), ('011', '110'), ('011', '111'), ('100', '000'), ('100', '001'), ('101', '010'), ('101', '011'), ('110', '100'), ('110', '101'), ('111', '110'), ('111', '111')]


In [7]:
G.add_nodes_from(nodes)
G.add_edges_from(edges)

In [8]:
# Optional. Run if you want an interactive graph drawing.
def plot_graph(G):
    net = Network(notebook=True, height='600px', width='600px', directed=True)
    net.from_nx(G)
    net.toggle_physics(False)
    net.show_buttons(filter_=['edges'])
    return net.show('example.html')
# plot_graph(G)

We find all the $k-cycle$ in above graph such that $k < n$.

In [9]:
cyc = list(nx.algorithms.simple_cycles(G))
for c in cyc:
    if len(c) < n:
        print(c)

['000']
['111']
['101', '010']


Out of all the possible words of length $n$. Only those that are parts of above cycles can be repeated.

We can see similar thing for $n=4$

In [10]:
n = 4
G = nx.DiGraph()
nodes = [''.join(tup) for tup in list(it.product(alphabet, repeat=n))]
edges = []
for node in nodes:
    for letter in alphabet:
        edges.append((node, node[1:]+letter))
G.add_nodes_from(nodes)
G.add_edges_from(edges)

In [11]:
print(nodes)

['0000', '0001', '0010', '0011', '0100', '0101', '0110', '0111', '1000', '1001', '1010', '1011', '1100', '1101', '1110', '1111']


In [12]:
print(edges)

[('0000', '0000'), ('0000', '0001'), ('0001', '0010'), ('0001', '0011'), ('0010', '0100'), ('0010', '0101'), ('0011', '0110'), ('0011', '0111'), ('0100', '1000'), ('0100', '1001'), ('0101', '1010'), ('0101', '1011'), ('0110', '1100'), ('0110', '1101'), ('0111', '1110'), ('0111', '1111'), ('1000', '0000'), ('1000', '0001'), ('1001', '0010'), ('1001', '0011'), ('1010', '0100'), ('1010', '0101'), ('1011', '0110'), ('1011', '0111'), ('1100', '1000'), ('1100', '1001'), ('1101', '1010'), ('1101', '1011'), ('1110', '1100'), ('1110', '1101'), ('1111', '1110'), ('1111', '1111')]


In [13]:
cyc = list(nx.algorithms.simple_cycles(G))
for c in cyc:
    if len(c) < n:
        print(c)

['0000']
['1111']
['1001', '0010', '0100']
['0101', '1010']
['1011', '0110', '1101']


We see that the words that part of $k-cycle$ such that $k<n$ are the only ones that can be repeated. Also, these are border words.

On of the construction of De Bruijn sequence involve finding a hamiltonian path in a $n$ dimensional De Bruijn graph. We can modify that construction to allow visiting the above mentioned vertices multiple time.

Alternately, we can make $n-1$ dimensional De Bruijn graph and find all the $k-cycle$ with $k \le n$ and instead of finding an Eulerian cycle that passes through all edges once, we find a circuit that passes through all edges exactly once except the edges that are part of mentioned $k-cycle$. Following is the example.

In [14]:
n = 3
G = nx.DiGraph()
nodes = [''.join(tup) for tup in list(it.product(alphabet, repeat=n))]
edges = []
for node in nodes:
    for letter in alphabet:
        edges.append((node, node[1:]+letter))
G.add_nodes_from(nodes)
G.add_edges_from(edges)

In [15]:
print(nodes)

['000', '001', '010', '011', '100', '101', '110', '111']


In [16]:
print(edges)

[('000', '000'), ('000', '001'), ('001', '010'), ('001', '011'), ('010', '100'), ('010', '101'), ('011', '110'), ('011', '111'), ('100', '000'), ('100', '001'), ('101', '010'), ('101', '011'), ('110', '100'), ('110', '101'), ('111', '110'), ('111', '111')]


In [17]:
def print_l_cycle(G, l):
    cyc = nx.algorithms.simple_cycles(G)
    for c in cyc:
        if len(c) <= l:
            print(c)
print_l_cycle(G, n)

['000']
['111']
['011', '110', '101']
['100', '001', '010']
['101', '010']


The notion of repeating edges can be thought of as having multiple edges. For this example what seem to be working is having $4$  edges for $000$ and $111$ cycle and $2$ for the rest of the cycles. So, we can add those edges.

In [18]:
G = nx.MultiDiGraph()
nodes = [''.join(tup) for tup in list(it.product(alphabet, repeat=n))]
edges = []
for node in nodes:
    for letter in alphabet:
        edges.append((node, node[1:]+letter))
G.add_nodes_from(nodes)
G.add_edges_from(edges)

more_edges = [('000', '000') for _ in range(3)]
G.add_edges_from(more_edges)
more_edges = [('111', '111') for _ in range(3)]
G.add_edges_from(more_edges)
v = ['001', '010', '100']
G.add_edges_from([(v[i], v[(i+1)%len(v)]) for i in range(len(v))])
v = ['011', '110', '101']
G.add_edges_from([(v[i], v[(i+1)%len(v)]) for i in range(len(v))])

[1, 1, 1]

In [19]:
print(G.edges)

[('000', '000', 0), ('000', '000', 1), ('000', '000', 2), ('000', '000', 3), ('000', '001', 0), ('001', '010', 0), ('001', '010', 1), ('001', '011', 0), ('010', '100', 0), ('010', '100', 1), ('010', '101', 0), ('011', '110', 0), ('011', '110', 1), ('011', '111', 0), ('100', '000', 0), ('100', '001', 0), ('100', '001', 1), ('101', '010', 0), ('101', '011', 0), ('101', '011', 1), ('110', '100', 0), ('110', '101', 0), ('110', '101', 1), ('111', '110', 0), ('111', '111', 0), ('111', '111', 1), ('111', '111', 2), ('111', '111', 3)]


In [20]:
# Exporting and using Cytoscape to view graph as repeated edges are overlapping in pyvis.
nx.write_gml(G, 'g.gml')

Edges are directed as given in previous graph, for some reason it's not rendering here.
Now we traverse this graph to find the longest trail (repeating vertices but not edges). If we look the corresponding word in paper.
$01010100100110110111111100000001$.
We see that it starts at $010$. And progresses as $010-10100100110110111111100000001$ Here each letter after $010$ indicates the last letter of the vertex we are approaching as per De Bruijn graph.

However, one thing I noticed is we can enter/exit any cycle exactly once. Meaning if we enter a cycle say at $010$ and we can traverce full cycle of $010-100-001$ at most twice, but if at any point, we get out of the cycle, then we cannot traverse a full cycle any more (partial is allowed) upon re-entry.

Example:
Let's start with $010$ we go to finish $010-100-001$ cycle twice, we get $010010010$. Here we have two instances of $0100$, $1001$ and $0010$, none disjoint.

Now, let's try another way, where we tranverse cycle once, to get $010010$, now instead of traversing cycle again, we exit and re-entre that cycle. This can be done by visiting $101$ once, so we get $01001010$. Now since we exit cycle once and came back, we cannot complete a full cycle. We can visit $100$ and $001$ to get $0100101001$, but cannot come back to $010$ as that will complete the cycle and the obtained word will be $01001010010$ with two disjoint instance of $0010$.

So, what I think can be done is we can construct a multiple edges graphs as mentioned and find the longest trail with restriction on cycles as mentioned above. I think that will give us the longest word with our properties. I tried looking for some algorithm that is polynomial time for such operations, so far I think there aren't any. But it's still looks better than brute-force.

I'm not sure at the moment if this will lead us to some breakthrough. I'm currently thinking about some ways to avoid exit/entry into cycle, in terms of graph, or maybe in some other way to translate this problem to something else where is might be easier to work with. Also I'll work over some alteration of Lyndon word construction of De Bruijn sequence, and see if there is something we can do.

In [21]:
n = 5
k = 2
G = nx.MultiDiGraph()
alphabet = [str(_) for _ in range(k)]
nodes = [''.join(tup) for tup in list(it.product(alphabet, repeat=n))]
G.add_nodes_from(nodes)
for node in nodes:
    for letter in alphabet:
        G.add_edge(node, node[1:]+letter, weight=node+letter)

In [22]:
print_l_cycle(G, n)

['00000']
['11111']
['00111', '01110', '11100', '11001', '10011']
['11001', '10011', '00110', '01100']
['01111', '11110', '11101', '11011', '10111']
['01110', '11101', '11011', '10111']
['00010', '00100', '01000', '10001']
['00010', '00100', '01000', '10000', '00001']
['00110', '01100', '11000', '10001', '00011']
['01010', '10101']
['01010', '10100', '01001', '10010', '00101']
['10110', '01101', '11011']
['10110', '01101', '11010', '10101', '01011']
['01001', '10010', '00100']
